In [14]:
import numpy as np

## __init__ vs __call__

init은 객체(모델하우스)를 생성할때 var = class()
call은 인스턴스(실제 집) 생성할때 private = var()


In [15]:
class Var():
    def __init__(self):
        print('init')
    def __call__(self):
        print('call')
        
a = Var()
b = a()

init
call


In [16]:
class Variable:
    def __init__(self,data):
        self.data = data

In [17]:
class Function:
    def __call__(self,input):
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [18]:
data = np.array(10)
x = Variable(data)
print(x.data)

10


In [19]:

x = Variable(np.array(10))
f = Function()
y = f(x) # Function의 __call__ 함수를 불러와서 연산했지만, 저장되는 데이터는 Variable class에

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [20]:
'''
Base class (기반클래스)를 Function으로 두고
구체적 기능은 Function을 기반으로 작업하기 위해 class edit
'''
class Function: #()가 있으면 달라질까? 달라지지는 않지만, Base class의 경우 상속하는 클래스가 없기때문에 ()가 없는 형태.
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self,x):
        raise NotImplementedError()
    
    
a = Function
a

__main__.Function

In [21]:
class Square(Function):
    def forward(self, x):
        return x**2

In [22]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [23]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [30]:
A = Square()
B = Exp()
C = Square()

x = (np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

TypeError: unsupported operand type(s) for ** or pow(): 'memoryview' and 'int'

In [32]:
def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps) #중앙 차분

In [38]:
f = Square()
x = Variable(np.array(2))
dy = numerical_diff(f,x)
print(type(dy),dy)

<class 'numpy.float64'> 4.000000000004


In [39]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))
x = Variable(np.array(.5))
dy = numerical_diff(f,x)
print(dy)


3.2974426293330694


## NumericalDifferentiation vs Backpropagation

**Numerical differentiation**  
- 장점 : 계산이 쉽다.
- 단점 : 메모리의 '자릿수 누락' 떄문에 오차가 생길 수 있다.  
계산량이 많아서 parameter가 많은 DL모델 특성 상 비효율적임.  

**Backpropagation**
- 장점 : 결과 값의 오차가 적다.  
연쇄법칙을 활용해 미분을 효율적으로 계산할 수 있다.
- 단점 : 알고리즘의 복잡성 때문에 버그 발생확률이 상대적으로 높다.  


*다만 NumericalDiff를 완전히 쓰지 않는 것이 아니라, BackPropagation의 결과 값을 검증하는 기울기 확인 때 비교 수치로 사용됨.*



In [42]:
class Variable:
    def __init__(self,data) -> None:
        self.data = data
        self.grad = None
        

class Function:
    def __call__(self,input) -> float:
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input #Backprop시 계산하기 위해 기억해야함.
        return output
        
    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()
        

In [43]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x * gy
        return gx
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [44]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(.5))
a = A(x)
b = B(a)
y = C(b)

In [47]:
y.grad = np.array(1.)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad) # numerical_diff : 3.2974426293330694


3.297442541400256


## automatic BackPropagation

In [48]:
class Variable:
    def __init__(self,data) -> None:
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self,func):
        self.creator = func
        

class Function:
    def __call__(self,input) -> float:
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output # 출력도 저장?
        return output
        
        
    def forward(self,x):
        raise NotImplementedError
    
    def backward(self,gy):
        raise NotImplementedError

SyntaxError: unexpected EOF while parsing (2085327003.py, line 1)